In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1026351,2021-02-23,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1026352,2021-02-24,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1026353,2021-02-25,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1026354,2021-02-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
27851,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27853,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27855,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27857,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
27859,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
635839,2021-02-23,Arkansas,Arkansas,5001,1949,38.00,5000,Arkansas,AR,Arkansas,State,3017804
635841,2021-02-24,Arkansas,Arkansas,5001,1963,38.00,5000,Arkansas,AR,Arkansas,State,3017804
635843,2021-02-25,Arkansas,Arkansas,5001,1967,38.00,5000,Arkansas,AR,Arkansas,State,3017804
635845,2021-02-26,Arkansas,Arkansas,5001,1971,38.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1026351,2021-02-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1026352,2021-02-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1026353,2021-02-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1026354,2021-02-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-02-27') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
403,2021-02-27,Snohomish,Washington,30828,538.00,53061,WA,County,822083,3749.99,65.44
804,2021-02-27,Cook,Illinois,473944,9906.00,17031,IL,County,5150233,9202.38,192.34
1204,2021-02-27,Orange,California,261022,3904.00,6059,CA,County,3175692,8219.37,122.93
1603,2021-02-27,Maricopa,Arizona,509830,9107.00,4013,AZ,County,4485414,11366.40,203.04
2002,2021-02-27,Los Angeles,California,1191521,21328.00,6037,CA,County,10039107,11868.79,212.45
...,...,...,...,...,...,...,...,...,...,...,...
1025927,2021-02-27,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1026065,2021-02-27,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1026172,2021-02-27,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1026275,2021-02-27,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
403,2021-02-27,Snohomish,Washington,30828,538.00,53061,WA,County,822083,3749.99,65.44,65.44,3749.99
804,2021-02-27,Cook,Illinois,473944,9906.00,17031,IL,County,5150233,9202.38,192.34,192.34,9202.38
1204,2021-02-27,Orange,California,261022,3904.00,6059,CA,County,3175692,8219.37,122.93,122.93,8219.37
1603,2021-02-27,Maricopa,Arizona,509830,9107.00,4013,AZ,County,4485414,11366.40,203.04,203.04,11366.40
2002,2021-02-27,Los Angeles,California,1191521,21328.00,6037,CA,County,10039107,11868.79,212.45,212.45,11868.79
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1025927,2021-02-27,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1026065,2021-02-27,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1026172,2021-02-27,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1026275,2021-02-27,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
723285,2021-02-27,Clarke,Alabama,3436,50.00,1025,AL,County,23622,14545.76,211.67,211.67,14545.76,1
664740,2021-02-27,Hale,Alabama,2094,68.00,1065,AL,County,14651,14292.54,464.13,464.13,14292.54,2
520581,2021-02-27,Lowndes,Alabama,1308,51.00,1085,AL,County,9726,13448.49,524.37,524.37,13448.49,3
461767,2021-02-27,Etowah,Alabama,13163,319.00,1055,AL,County,102268,12871.08,311.93,311.93,12871.08,4
387696,2021-02-27,Franklin,Alabama,3989,77.00,1059,AL,County,31362,12719.21,245.52,245.52,12719.21,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
975966,2021-02-27,Platte,Wyoming,589,11.00,56031,WY,County,8393,7017.75,131.06,131.06,7017.75,19
634839,2021-02-27,Sublette,Wyoming,675,7.00,56035,WY,County,9831,6866.04,71.20,71.20,6866.04,20
876403,2021-02-27,Niobrara,Wyoming,149,2.00,56027,WY,County,2356,6324.28,84.89,84.89,6324.28,21
788027,2021-02-27,Lincoln,Wyoming,1224,11.00,56023,WY,County,19830,6172.47,55.47,55.47,6172.47,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
520581,2021-02-27,Lowndes,Alabama,1308,51.00,1085,AL,County,9726,13448.49,524.37,524.37,13448.49,3,1
664740,2021-02-27,Hale,Alabama,2094,68.00,1065,AL,County,14651,14292.54,464.13,464.13,14292.54,2,2
460407,2021-02-27,Clay,Alabama,1438,54.00,1027,AL,County,13235,10865.13,408.01,408.01,10865.13,21,3
221952,2021-02-27,Walker,Alabama,6514,255.00,1127,AL,County,63521,10254.88,401.44,401.44,10254.88,32,4
561258,2021-02-27,Greene,Alabama,875,32.00,1063,AL,County,8111,10787.82,394.53,394.53,10787.82,23,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
559905,2021-02-27,Hot Springs,Wyoming,360,3.00,56017,WY,County,4413,8157.72,67.98,67.98,8157.72,17,19
768498,2021-02-27,Uinta,Wyoming,2054,12.00,56041,WY,County,20226,10155.25,59.33,59.33,10155.25,4,20
788027,2021-02-27,Lincoln,Wyoming,1224,11.00,56023,WY,County,19830,6172.47,55.47,55.47,6172.47,22,21
220914,2021-02-27,Teton,Wyoming,3348,9.00,56039,WY,County,23464,14268.67,38.36,38.36,14268.67,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,65.44,3749.99,14,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023929,2021-02-23,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1023930,2021-02-24,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1023931,2021-02-25,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3
1023932,2021-02-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,3


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
720531,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,211.67,14545.76,46,1,2.00
720532,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,211.67,14545.76,46,1,0.00
720533,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,211.67,14545.76,46,1,1.00
720534,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,211.67,14545.76,46,1,4.00
720535,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,211.67,14545.76,46,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
801111,2021-02-23,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
801112,2021-02-24,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
801113,2021-02-25,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00
801114,2021-02-26,Crook,Wyoming,417,11.00,56011,WY,County,7584,5498.42,145.04,145.04,5498.42,6,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
518495,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13448.49,1,3,1.00,0.00
518496,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13448.49,1,3,0.00,0.00
518497,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13448.49,1,3,0.00,0.00
518498,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13448.49,1,3,0.00,0.00
518499,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,524.37,13448.49,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
516795,2021-02-23,Albany,Wyoming,3892,11.00,56001,WY,County,38880,10010.29,28.29,28.29,10033.44,23,6,-1.00,0.00
516796,2021-02-24,Albany,Wyoming,3893,11.00,56001,WY,County,38880,10012.86,28.29,28.29,10033.44,23,6,1.00,0.00
516797,2021-02-25,Albany,Wyoming,3894,11.00,56001,WY,County,38880,10015.43,28.29,28.29,10033.44,23,6,1.00,0.00
516798,2021-02-26,Albany,Wyoming,3901,11.00,56001,WY,County,38880,10033.44,28.29,28.29,10033.44,23,6,7.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-02-27') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
264824,2021-02-27,Imperial,California,26888,590.00,6025,CA,County,181215,14837.62,325.58,325.58,14837.62,1,2,33.00,4.00
2002,2021-02-27,Los Angeles,California,1191521,21328.00,6037,CA,County,10039107,11868.79,212.45,212.45,11868.79,2,5,1662.00,87.00
463079,2021-02-27,Inyo,California,1271,37.00,6027,CA,County,18039,7045.85,205.11,205.11,7045.85,3,27,7.00,1.00
59193,2021-02-27,Stanislaus,California,56024,946.00,6099,CA,County,550660,10173.97,171.79,171.79,10173.97,4,10,137.00,0.00
73350,2021-02-27,Tulare,California,47869,758.00,6107,CA,County,466195,10268.02,162.59,162.59,10268.02,5,9,0.00,0.00
27492,2021-02-27,Riverside,California,289450,3767.00,6065,CA,County,2470546,11716.03,152.48,152.48,11716.03,6,6,0.00,0.00
47125,2021-02-27,San Joaquin,California,66709,1101.00,6077,CA,County,762148,8752.76,144.46,144.46,8752.76,7,18,0.00,0.00
26776,2021-02-27,Fresno,California,95202,1443.00,6019,CA,County,999101,9528.77,144.43,144.43,9528.77,8,14,187.00,21.00
419789,2021-02-27,Merced,California,28971,397.00,6047,CA,County,277680,10433.23,142.97,142.97,10433.23,9,8,0.00,0.00
561200,2021-02-27,Kings,California,22042,218.00,6031,CA,County,152940,14412.19,142.54,142.54,14412.19,10,3,39.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
975517,2021-02-27,Lassen,California,5601,19.00,6035,CA,County,30573,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00
264824,2021-02-27,Imperial,California,26888,590.00,6025,CA,County,181215,14837.62,325.58,325.58,14837.62,1,2,33.00,4.00
561200,2021-02-27,Kings,California,22042,218.00,6031,CA,County,152940,14412.19,142.54,142.54,14412.19,10,3,39.00,0.00
132110,2021-02-27,San Bernardino,California,286291,2816.00,6071,CA,County,2180085,13132.10,129.17,129.17,13132.10,12,4,214.00,33.00
2002,2021-02-27,Los Angeles,California,1191521,21328.00,6037,CA,County,10039107,11868.79,212.45,212.45,11868.79,2,5,1662.00,87.00
27492,2021-02-27,Riverside,California,289450,3767.00,6065,CA,County,2470546,11716.03,152.48,152.48,11716.03,6,6,0.00,0.00
166005,2021-02-27,Kern,California,103143,827.00,6029,CA,County,900202,11457.76,91.87,91.87,11457.76,25,7,145.00,1.00
419789,2021-02-27,Merced,California,28971,397.00,6047,CA,County,277680,10433.23,142.97,142.97,10433.23,9,8,0.00,0.00
73350,2021-02-27,Tulare,California,47869,758.00,6107,CA,County,466195,10268.02,162.59,162.59,10268.02,5,9,0.00,0.00
59193,2021-02-27,Stanislaus,California,56024,946.00,6099,CA,County,550660,10173.97,171.79,171.79,10173.97,4,10,137.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
264824,2021-02-27,Imperial,California,26888,590.00,6025,CA,County,181215,14837.62,325.58,325.58,14837.62,1,2,33.00,4.00
2002,2021-02-27,Los Angeles,California,1191521,21328.00,6037,CA,County,10039107,11868.79,212.45,212.45,11868.79,2,5,1662.00,87.00
463079,2021-02-27,Inyo,California,1271,37.00,6027,CA,County,18039,7045.85,205.11,205.11,7045.85,3,27,7.00,1.00
59193,2021-02-27,Stanislaus,California,56024,946.00,6099,CA,County,550660,10173.97,171.79,171.79,10173.97,4,10,137.00,0.00
73350,2021-02-27,Tulare,California,47869,758.00,6107,CA,County,466195,10268.02,162.59,162.59,10268.02,5,9,0.00,0.00
27492,2021-02-27,Riverside,California,289450,3767.00,6065,CA,County,2470546,11716.03,152.48,152.48,11716.03,6,6,0.00,0.00
47125,2021-02-27,San Joaquin,California,66709,1101.00,6077,CA,County,762148,8752.76,144.46,144.46,8752.76,7,18,0.00,0.00
26776,2021-02-27,Fresno,California,95202,1443.00,6019,CA,County,999101,9528.77,144.43,144.43,9528.77,8,14,187.00,21.00
419789,2021-02-27,Merced,California,28971,397.00,6047,CA,County,277680,10433.23,142.97,142.97,10433.23,9,8,0.00,0.00
561200,2021-02-27,Kings,California,22042,218.00,6031,CA,County,152940,14412.19,142.54,142.54,14412.19,10,3,39.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,325.58,14837.62,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,325.58,14837.62,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,325.58,14837.62,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,325.58,14837.62,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,325.58,14837.62,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3462,02/23/21,Lassen,5594,19.00,18297.19,62.15,62.15,18320.09,38,1,3.00,0.00
3463,02/24/21,Lassen,5594,19.00,18297.19,62.15,62.15,18320.09,38,1,0.00,0.00
3464,02/25/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,38,1,7.00,0.00
3465,02/26/21,Lassen,5601,19.00,18320.09,62.15,62.15,18320.09,38,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,325.58,14837.62,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,325.58,14837.62,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,325.58,14837.62,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,325.58,14837.62,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,325.58,14837.62,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3536,02/23/21,Kings,21916,218.00,14329.80,142.54,142.54,14412.19,10,3,35.00,2.00
3537,02/24/21,Kings,21916,218.00,14329.80,142.54,142.54,14412.19,10,3,0.00,0.00
3538,02/25/21,Kings,21951,218.00,14352.69,142.54,142.54,14412.19,10,3,35.00,0.00
3539,02/26/21,Kings,22003,218.00,14386.69,142.54,142.54,14412.19,10,3,52.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)